In [1]:
# So the idea here is to do the word embedding 
# using a pre-trained model provided by TF Hub
# which is similar to Docker hub for models

from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import tensorflow as tf
!pip install -q tensorflow-hub
!pip install -q tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Make sure tf version is 2 otherwise shit will break!
# Also, I'm running this on Python3
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT available")

Version:  2.0.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT available


In [2]:
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews",
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True
)

In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch.shape

TensorShape([10])

In [4]:
train_labels_batch

<tf.Tensor: id=220, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=400, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(
    train_data.shuffle(10000).batch(512), 
    epochs=20, 
    validation_data=validation_data.batch(512),
    verbose=1
)

Epoch 1/20
30/30 [==============================] - 4s 136ms/step - loss: 0.7021 - accuracy: 0.5762 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 3s 111ms/step - loss: 0.6250 - accuracy: 0.6561 - val_loss: 0.6078 - val_accuracy: 0.6710
Epoch 3/20
30/30 [==============================] - 3s 113ms/step - loss: 0.5902 - accuracy: 0.6962 - val_loss: 0.5786 - val_accuracy: 0.7006
Epoch 4/20
30/30 [==============================] - 3s 115ms/step - loss: 0.5566 - accuracy: 0.7247 - val_loss: 0.5507 - val_accuracy: 0.7254
Epoch 5/20
30/30 [==============================] - 3s 113ms/step - loss: 0.5270 - accuracy: 0.7516 - val_loss: 0.5216 - val_accuracy: 0.7516
Epoch 6/20
30/30 [==============================] - 4s 118ms/step - loss: 0.4942 - accuracy: 0.7774 - val_loss: 0.4906 - val_accuracy: 0.7730
Epoch 7/20
30/30 [==============================] - 4s 117ms/step - loss: 0.4524 - accuracy: 0.7988 - val_loss: 0.4609 - val_accuracy: 0.789

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3247 - accuracy: 0.8596
loss: 0.325
accuracy: 0.860
